# Data EDA

## Import Packages

In [65]:
import os
import boto3 # AWS
import pandas as pd
from pandas import json_normalize
import numpy as np
import json

# Set the maximum number of rows to 200
pd.set_option("display.max_rows", 200)
import pandas as pd

# Set the maximum number of columns to 200
pd.set_option('display.max_columns', 200)

## Import Data from AWS Bucket

In [23]:
# Creating an S3 client object
s3 = boto3.client('s3')

# Specifying the name of the bucket
bucket_name = 'capstonehaystacks'

# List of CSV files to download
files = [
    'atlanta_cbsa_zip.csv',
    'core_geo_dataset.csv',
    'crime_rating_zipcode.csv',
    'elementary_schools.csv',
    'GA_LISTINGS_SALES_V2.csv',
    'GA_LISTINGS_SALES.csv',
    'high_schools.csv',
    'middle_schools.csv',
    'all_zips_grocery_store.json',
    'all_zips_restaurant.json'
]

# Dictionary to store the dataframes
dataframes = {}

# Downloading the files from S3 and parsing them into data
for file_name in files:
    _, file_ext = os.path.splitext(file_name)
    if file_ext == '.csv':
        s3.download_file(bucket_name, file_name, file_name)
        dataframes[file_name[:-4]] = pd.read_csv(file_name, index_col=False)
    elif file_ext == '.json':
        response = s3.get_object(Bucket=bucket_name, Key=file_name)
        json_content = response['Body'].read().decode('utf-8')
        dataframes[file_name[:-5]] = json.loads(json_content)

In [37]:
# Access the dataframes using their respective keys
grocery = dataframes['all_zips_grocery_store']
restaurant = dataframes['all_zips_restaurant']
atlanta_geo = dataframes['atlanta_cbsa_zip']
geo = dataframes['core_geo_dataset']
crime = dataframes['crime_rating_zipcode']
elem_school = dataframes['elementary_schools']
high_school = dataframes['high_schools']
mid_school = dataframes['middle_schools']
GA_listing = dataframes['GA_LISTINGS_SALES_V2']

## Data Cleanup

In [38]:
# Converting .csv's to DataFrames
data = [atlanta_geo, geo, crime, elem_school, high_school, mid_school, GA_listing]

for i in data:
    pd.DataFrame(i)

In [39]:
# Converting JSON to DataFrames & Flattening the data
flattened_grocery = json_normalize(grocery, record_path=["responce", "results"], meta=[["index"], ["census_zcta_geoid"], ["params", "key"], ["params", "location"], ["params", "radius"], ["params", "type"], ["responce", "status"]], errors="ignore")
flattened_restaurant = json_normalize(restaurant, record_path=["responce", "results"], meta=[["index"], ["census_zcta_geoid"], ["params", "key"], ["params", "location"], ["params", "radius"], ["params", "type"], ["responce", "status"]], errors="ignore")

grocery = pd.DataFrame(flattened_grocery)
restaurant = pd.DataFrame(flattened_restaurant)

In [60]:
print('Grocery: ', grocery.shape)
print('\n', grocery.dtypes)
print('\n','-'*50,'\n')
print('Restuarant: ', restaurant.shape)
print('\n',restaurant.dtypes)

Grocery:  (402803, 31)

 icon                                object
icon_background_color               object
icon_mask_base_uri                  object
name                                object
place_id                            object
reference                           object
scope                               object
types                               object
vicinity                            object
geometry.location.lat              float64
geometry.location.lng              float64
geometry.viewport.northeast.lat    float64
geometry.viewport.northeast.lng    float64
geometry.viewport.southwest.lat    float64
geometry.viewport.southwest.lng    float64
business_status                     object
plus_code.compound_code             object
plus_code.global_code               object
permanently_closed                  object
photos                              object
rating                             float64
user_ratings_total                 float64
opening_hours.open_now       

In [61]:
# Converting the "census_zcta_geoid" column to integers
grocery["census_zcta_geoid"] = grocery["census_zcta_geoid"].astype(int)
restaurant["census_zcta_geoid"] = restaurant["census_zcta_geoid"].astype(int)

In [69]:
# Using the zip codes in this file to filter the grocery_df
zip_list = atlanta_geo["census_zcta5_geoid"].tolist() # works for both

# Creating a boolean mask
mask_grocery = grocery["census_zcta_geoid"].isin(zip_list)
mask_rest = restaurant["census_zcta_geoid"].isin(zip_list)

# Filtering the dataframe based on the boolean mask
grocery_atlanta = grocery[mask_grocery]
restaurant_atlanta = restaurant[mask_rest]

In [67]:
# Dropping columns that arent useful in grocery
grocery_atlanta = grocery_atlanta.drop(['icon', 'icon_background_color', 'params.location', 
                                              'params.key', 'params.radius', 'params.type', 'icon_mask_base_uri', 
                                              'reference', 'scope', 'index', 'responce.status'], 
                                             axis=1).reset_index(drop=True)

In [70]:
# Dropping columns that arent useful and resetting index in restatuarant
restaurant_atlanta = restaurant_atlanta.drop(['icon', 'icon_background_color', 'params.location',
                                              'params.key', 'params.radius', 'params.type', 'icon_mask_base_uri',
                                              'reference', 'scope', 'index', 'responce.status'], 
                                             axis=1).reset_index(drop=True)

In [72]:
# Combining the two dataframes by concating vertically into a Point of Interest DF
POI = pd.concat([restaurant_atlanta, grocery_atlanta], ignore_index=True)

In [73]:
# Checking or duplicates
POI.duplicated(subset=['place_id']).sum()

1075

In [74]:
#Dropping duplicates
POI.drop_duplicates(subset='place_id', inplace=True)

In [75]:
POI.shape

(5213, 31)

In [ ]:
# Saving to AWS
file_name2 = "points-of-interest-haystacks.csv"
bucket_name = 'capstonehaystacks'

# Upload the file to S3
s3.upload_file(file_name2, bucket_name, file_name2)

## Summary statistics and Checking for missing values

### grocery

In [41]:
print("Shape of grocery data: ", grocery.shape)
grocery.head()

Shape of grocery data:  (402803, 31)


,icon,icon_background_color,icon_mask_base_uri,name,place_id,reference,scope,types,vicinity,geometry.location.lat,...,user_ratings_total,opening_hours.open_now,price_level,index,census_zcta_geoid,params.key,params.location,params.radius,params.type,responce.status
0,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Alert,ChIJrUteIPmra4gRTUg6zPdb-U0,ChIJrUteIPmra4gRTUg6zPdb-U0,GOOGLE,"[locality, political]",Alert,39.160330,...,NaN,NaN,NaN,0,47236,NaN,NaN,NaN,NaN,OK
1,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Bear Creek Baptist Church Cemetery,ChIJA9RmuJ2ua4gRIH78FSmGOJY,ChIJA9RmuJ2ua4gRIH78FSmGOJY,GOOGLE,"[cemetery, point_of_interest, establishment]","3945 West 1000 North, Westport",39.130229,...,NaN,NaN,NaN,0,47236,NaN,NaN,NaN,NaN,OK
2,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Miller Enterprises,ChIJsz8OSlCua4gRmlx2iahYGfQ,ChIJsz8OSlCua4gRmlx2iahYGfQ,GOOGLE,"[point_of_interest, establishment]","5585 West Private Road 850 North, Scipio",39.108485,...,NaN,NaN,NaN,0,47236,NaN,NaN,NaN,NaN,OK
3,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Rockcreek Elementary School,ChIJIW-2WxCma4gRQh4kHuMkpaY,ChIJIW-2WxCma4gRQh4kHuMkpaY,GOOGLE,"[primary_school, school, point_of_interest, es...","13000 East 200 South, Columbus",39.174729,...,16.0,NaN,NaN,0,47236,NaN,NaN,NaN,NaN,OK
4,https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Seymour Transport Inc,ChIJ7YB3Z9aua4gRmjX6GeAEY7g,ChIJ7YB3Z9aua4gRmjX6GeAEY7g,GOOGLE,"[moving_company, point_of_interest, establishm...","18375 East 345 South, Elizabethtown",39.152817,...,NaN,NaN,NaN,0,47236,NaN,NaN,NaN,NaN,OK


In [42]:
print("Summary statistics of grocery data: ")
grocery.describe()

Summary statistics of grocery data: 


,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,rating,user_ratings_total,price_level
count,402803.000000,402803.000000,402803.000000,402803.000000,402803.000000,402803.000000,177115.000000,177115.000000,36621.00000
mean,38.561179,-88.127154,38.565837,-88.121466,38.556636,-88.132769,4.253026,303.074872,1.53087
std,5.317502,13.956969,5.317130,13.956141,5.317763,13.957215,0.779002,1640.236342,0.62333
min,-14.348378,-176.666079,-14.345637,-176.664852,-14.351737,-176.713098,1.000000,1.000000,0.00000
25%,35.214905,-94.220000,35.220433,-94.216345,35.210718,-94.224471,4.000000,3.000000,1.00000
50%,39.436647,-86.677678,39.441737,-86.668663,39.432992,-86.683590,4.400000,15.000000,1.00000
75%,42.171158,-79.163096,42.174812,-79.160292,42.166994,-79.167538,4.900000,138.000000,2.00000
max,71.298056,145.772106,71.339755,145.773638,71.294478,145.770940,5.000000,233292.000000,4.00000


In [43]:
print("Missing values in grocery data: ")
grocery.isnull().sum()

Missing values in grocery data: 


icon                                    0
icon_background_color                   0
icon_mask_base_uri                      0
name                                    0
place_id                                0
reference                               0
scope                                   0
types                                   0
vicinity                              355
geometry.location.lat                   0
geometry.location.lng                   0
geometry.viewport.northeast.lat         0
geometry.viewport.northeast.lng         0
geometry.viewport.southwest.lat         0
geometry.viewport.southwest.lng         0
business_status                     38291
plus_code.compound_code             44788
plus_code.global_code               44679
permanently_closed                 373902
photos                             291246
rating                             225688
user_ratings_total                 225688
opening_hours.open_now             212198
price_level                       

### restaurant

In [44]:
print("Shape of resaturant data: ", restaurant.shape)
restaurant.head()

Shape of resaturant data:  (198343, 31)


,business_status,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,reference,...,plus_code.compound_code,plus_code.global_code,permanently_closed,index,census_zcta_geoid,params.key,params.location,params.radius,params.type,responce.status
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Montgomery's Steakhouse,"[{'height': 1800, 'html_attributions': ['<a hr...",ChIJM8M15Z_NaogRhWPRBY0U9Ms,2.0,3.9,ChIJM8M15Z_NaogRhWPRBY0U9Ms,...,"RHWJ+JC Spiceland, IN, USA",86FPRHWJ+JC,NaN,3,47337,NaN,NaN,NaN,NaN,OK
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Casey's,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJ436SF2bNaogRg4KeIf0HP4I,1.0,4.7,ChIJ436SF2bNaogRg4KeIf0HP4I,...,"RHP8+95 Spiceland, IN, USA",86FPRHP8+95,NaN,3,47337,NaN,NaN,NaN,NaN,OK
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,pizza king,NaN,ChIJrXgFGZ_NaogRFRoBLNjIS2U,2.0,NaN,ChIJrXgFGZ_NaogRFRoBLNjIS2U,...,"RHWJ+G9 Spiceland, IN, USA",86FPRHWJ+G9,NaN,3,47337,NaN,NaN,NaN,NaN,OK
3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Rasoi Indian Kitchen,"[{'height': 6936, 'html_attributions': ['<a hr...",ChIJ9VrB1Y7NaogRw4kONdkCOLE,NaN,4.6,ChIJ9VrB1Y7NaogRw4kONdkCOLE,...,"RHXH+9W Spiceland, IN, USA",86FPRHXH+9W,NaN,3,47337,NaN,NaN,NaN,NaN,OK
4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Denny's,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJPXFl0gvNaogR8zPNCBnm7O8,1.0,3.9,ChIJPXFl0gvNaogR8zPNCBnm7O8,...,"VH2P+QP Spiceland, IN, USA",86FPVH2P+QP,NaN,3,47337,NaN,NaN,NaN,NaN,OK


In [45]:
print("Summary statistics of restaurant data: ")
restaurant.describe()

Summary statistics of restaurant data: 


,price_level,rating,user_ratings_total,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng
count,158284.000000,193013.000000,193013.000000,198343.000000,198343.000000,198343.000000,198343.000000,198343.000000,198343.000000
mean,1.501358,4.163076,636.879210,37.675969,-89.182384,37.677322,-89.181025,37.674612,-89.183744
std,0.604327,0.458927,1030.388945,5.222885,16.102284,5.222883,16.102281,5.222887,16.102286
min,0.000000,1.000000,1.000000,-14.338562,-170.750152,-14.336844,-170.748756,-14.339541,-170.751454
25%,1.000000,3.900000,109.000000,34.101708,-95.833071,34.102984,-95.831758,34.100286,-95.834456
50%,1.000000,4.200000,310.000000,38.968308,-85.013824,38.969773,-85.012725,38.967075,-85.015423
75%,2.000000,4.500000,783.000000,41.183341,-77.608131,41.184651,-77.606809,41.181953,-77.609507
max,4.000000,5.000000,44144.000000,71.297957,145.771507,71.299473,145.772856,71.296775,145.770158


In [46]:
print("Missing values in restaurant data: ")
restaurant.isnull().sum()

Missing values in restaurant data: 


business_status                       103
icon                                    0
icon_background_color                   0
icon_mask_base_uri                      0
name                                    0
photos                              56875
place_id                                0
price_level                         40059
rating                               5330
reference                               0
scope                                   0
types                                   0
user_ratings_total                   5330
vicinity                                0
geometry.location.lat                   0
geometry.location.lng                   0
geometry.viewport.northeast.lat         0
geometry.viewport.northeast.lng         0
geometry.viewport.southwest.lat         0
geometry.viewport.southwest.lng         0
opening_hours.open_now              13127
plus_code.compound_code               550
plus_code.global_code                 549
permanently_closed                

### geo

In [11]:
print("Shape of geo data: ", geo.shape)
geo.head()

Shape of geo data:  (136473, 7)


,Unnamed: 0,census_zcta5_geoid,census_zcta5_lat,census_zcta5_lon,census_tract_geoid,census_tract_lat,census_tract_lon
0,0,47236,39.151743,-85.725277,18005011300,39.20876,-85.760188
1,1,47283,39.170024,-85.587896,18005011300,39.20876,-85.760188
2,2,47203,39.230418,-85.832263,18005011300,39.20876,-85.760188
3,3,47232,39.110924,-85.790836,18005011300,39.20876,-85.760188
4,4,47201,39.148221,-85.999407,18005011300,39.20876,-85.760188


In [12]:
print("Summary statistics of geo data: ")
geo.describe()

Summary statistics of geo data: 


,Unnamed: 0,census_zcta5_geoid,census_zcta5_lat,census_zcta5_lon,census_tract_geoid,census_tract_lat,census_tract_lon
count,136473.000000,136473.000000,136473.000000,136473.000000,1.364730e+05,136473.000000,136473.000000
mean,68236.000000,48801.475024,38.257041,-90.396386,2.921522e+10,38.256722,-90.382960
std,39396.505981,28196.921980,5.254177,15.114411,1.592141e+10,5.252617,15.190555
min,0.000000,601.000000,17.724710,-176.668632,1.001020e+09,17.694722,-166.770979
25%,34118.000000,25169.000000,34.778047,-96.803030,1.703597e+10,34.774634,-96.798224
50%,68236.000000,47946.000000,39.366217,-86.767472,2.912396e+10,39.366052,-86.763794
75%,102354.000000,72677.000000,41.693960,-79.847468,4.201101e+10,41.691306,-79.847487
max,136472.000000,99929.000000,71.253861,-64.708215,7.803096e+10,71.287519,178.338813


In [13]:
print("Missing values in geo data: ")
geo.isnull().sum()

Missing values in geo data: 


Unnamed: 0            0
census_zcta5_geoid    0
census_zcta5_lat      0
census_zcta5_lon      0
census_tract_geoid    0
census_tract_lat      0
census_tract_lon      0
dtype: int64

### atlanta_geo

In [14]:
print("Shape of atlanta_geo data: ", atlanta_geo.shape)
atlanta_geo.head()

Shape of atlanta_geo data:  (241, 9)


,Unnamed: 0,census_cbsa_geoid,census_cbsa_name,census_cbsa_lsad,census_cbsa_lat,census_cbsa_lon,census_zcta5_geoid,census_zcta5_lat,census_zcta5_lon
0,3208,12060,"Atlanta-Sandy Springs-Alpharetta, GA",M1,33.693728,-84.399911,30309,33.799851,-84.385837
1,3219,12060,"Atlanta-Sandy Springs-Alpharetta, GA",M1,33.693728,-84.399911,30517,34.130296,-83.797289
2,3222,12060,"Atlanta-Sandy Springs-Alpharetta, GA",M1,33.693728,-84.399911,30548,34.089396,-83.763599
3,3225,12060,"Atlanta-Sandy Springs-Alpharetta, GA",M1,33.693728,-84.399911,30518,34.133822,-84.022599
4,3227,12060,"Atlanta-Sandy Springs-Alpharetta, GA",M1,33.693728,-84.399911,30334,33.748652,-84.387894


In [15]:
print("Summary statistics of atlanta_geo data: ")
atlanta_geo.describe()

Summary statistics of atlanta_geo data: 


,Unnamed: 0,census_cbsa_geoid,census_cbsa_lat,census_cbsa_lon,census_zcta5_geoid,census_zcta5_lat,census_zcta5_lon
count,241.000000,241.0,2.410000e+02,2.410000e+02,241.000000,241.000000,241.000000
mean,4411.946058,12060.0,3.369373e+01,-8.439991e+01,30268.431535,33.742742,-84.365478
std,749.683606,0.0,1.352841e-13,1.851256e-13,277.487365,0.367733,0.398738
min,3208.000000,12060.0,3.369373e+01,-8.439991e+01,30002.000000,32.829319,-85.304390
25%,3708.000000,12060.0,3.369373e+01,-8.439991e+01,30096.000000,33.539118,-84.591033
50%,4486.000000,12060.0,3.369373e+01,-8.439991e+01,30218.000000,33.773335,-84.373997
75%,5072.000000,12060.0,3.369373e+01,-8.439991e+01,30317.000000,33.953141,-84.145978
max,5685.000000,12060.0,3.369373e+01,-8.439991e+01,31830.000000,34.728127,-83.183971


In [16]:
print("Missing values in atlanta_geo data: ")
atlanta_geo.isnull().sum()

Missing values in atlanta_geo data: 


Unnamed: 0            0
census_cbsa_geoid     0
census_cbsa_name      0
census_cbsa_lsad      0
census_cbsa_lat       0
census_cbsa_lon       0
census_zcta5_geoid    0
census_zcta5_lat      0
census_zcta5_lon      0
dtype: int64

### crime

In [17]:
print("Shape of crime data: ", crime.shape)
crime.head()

Shape of crime data:  (726, 27)


,Unnamed: 0,census_state_abbr,census_zcta5_geoid,census_cbsa_geoid_count,census_cbsa_geoid_list,overall_crime_grade,violent_crime_grade,violent_crime_total_rate,violent_crime_assault_rate,violent_crime_robbery_rate,...,property_crime_arson_rate,other_crime_grade,other_crime_total_rate,other_crime_kidnapping_rate,other_crime_drug_crimes_rate,other_crime_vandalism_rate,other_crime_identity_theft_rate,other_crime_animal_cruelty_rate,crime_safety_paragraph,interpreting_crime_map_paragraph
0,13719,GA,30002,1,12060,D-,C,3.106,1.7060,1.0670,...,0.1075,B+,5.987,0.1236,0.9456,3.981,0.8824,0.0542,The D- grade means the rate of crime is much h...,"When looking at the crime map for 30002, remem..."
1,13720,GA,30004,1,12060,C-,A-,1.553,0.9140,0.3858,...,0.1017,A+,3.171,0.0404,0.7863,1.524,0.8159,0.0042,The C- grade means the rate of crime is slight...,"When looking at the crime map for 30004, remem..."
2,13721,GA,30005,1,12060,B+,A-,1.480,0.7977,0.4288,...,0.1118,A+,2.709,0.0512,0.3942,1.394,0.8653,0.0041,The B+ grade means the rate of crime is lower ...,"When looking at the crime map for 30005, remem..."
3,13722,GA,30008,1,12060,D+,C,2.910,1.2040,1.3940,...,0.0946,B-,7.648,0.0883,0.9090,5.777,0.8214,0.0522,The D+ grade means the rate of crime is higher...,"When looking at the crime map for 30008, remem..."
4,13723,GA,30009,1,12060,D-,A-,1.611,1.0460,0.2950,...,0.1224,A,4.314,0.0453,0.8082,2.565,0.8910,0.0044,The D- grade means the rate of crime is much h...,"When looking at the crime map for 30009, remem..."


In [18]:
print("Summary statistics of crime data: ")
crime.describe()

Summary statistics of crime data: 


,Unnamed: 0,census_zcta5_geoid,census_cbsa_geoid_count,violent_crime_total_rate,violent_crime_assault_rate,violent_crime_robbery_rate,violent_crime_rape_rate,violent_crime_murder_rate,property_crime_total_rate,property_crime_theft_rate,property_crime_vehicle_theft_rate,property_crime_burglary_rate,property_crime_arson_rate,other_crime_total_rate,other_crime_kidnapping_rate,other_crime_drug_crimes_rate,other_crime_vandalism_rate,other_crime_identity_theft_rate,other_crime_animal_cruelty_rate
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,725.000000,726.000000,726.000000,725.000000,726.000000,726.000000
mean,14081.500000,31192.415978,1.057851,3.628924,2.415621,0.909414,0.256657,0.047235,30.323595,21.552821,2.232393,6.452248,0.086279,10.496128,0.158867,2.227102,7.233889,0.837094,0.043337
std,209.722436,1886.384965,0.602947,2.636762,2.004043,0.972578,0.060357,0.078517,28.738825,24.986148,2.807408,6.357703,0.078180,10.331241,0.222260,2.257033,9.603389,0.156147,0.041284
min,13719.000000,30002.000000,0.000000,0.943800,0.077100,0.009800,0.220900,0.001000,5.376000,0.611900,0.571400,1.704000,0.019300,1.960000,0.004900,0.006000,0.168400,0.733200,0.002100
25%,13900.250000,30315.250000,1.000000,2.210250,1.394250,0.309775,0.241725,0.025400,17.155000,10.130000,1.132000,4.570000,0.056725,5.755000,0.050225,0.900700,3.363000,0.750025,0.008525
50%,14081.500000,30723.000000,1.000000,2.911500,1.936500,0.622250,0.250000,0.030250,22.940000,14.805000,1.532000,5.796500,0.074350,8.380000,0.080950,1.516000,5.303000,0.785000,0.032700
75%,14262.750000,31515.250000,1.000000,4.283750,2.838500,1.195750,0.261375,0.039550,33.357500,24.150000,2.246500,6.846000,0.094975,12.530000,0.156800,2.755750,8.589000,0.868625,0.063000
max,14444.000000,39897.000000,4.000000,34.390000,29.960000,15.250000,1.706000,1.145000,465.300000,402.000000,24.050000,101.100000,1.670000,189.400000,2.568000,16.620000,186.500000,3.055000,0.229300


In [19]:
print("Missing values in crime data: ")
crime.isnull().sum()

Missing values in crime data: 


Unnamed: 0                             0
census_state_abbr                      0
census_zcta5_geoid                     0
census_cbsa_geoid_count                0
census_cbsa_geoid_list                 0
overall_crime_grade                    0
violent_crime_grade                    0
violent_crime_total_rate               0
violent_crime_assault_rate             0
violent_crime_robbery_rate             0
violent_crime_rape_rate                0
violent_crime_murder_rate              0
property_crime_grade                   0
property_crime_total_rate              0
property_crime_theft_rate              0
property_crime_vehicle_theft_rate      0
property_crime_burglary_rate           0
property_crime_arson_rate              0
other_crime_grade                      1
other_crime_total_rate                 1
other_crime_kidnapping_rate            0
other_crime_drug_crimes_rate           0
other_crime_vandalism_rate             1
other_crime_identity_theft_rate        0
other_crime_anim

### elem_school

In [20]:
print("Shape of elem_school data: ", elem_school.shape)
elem_school.head()

Shape of elem_school data:  (2719, 16)


,Unnamed: 0,id,districtID,districtName,districtCity,lat,long,name,gradeLevels,address,rating,ratingScale,schoolType,zipcode,studentsPerTeacher,parentRating
0,0,1038,73,Fulton County School District,Atlanta,34.071564,-84.363319,Sweet Apple Elementary School,PK-5,"12025 Etris Road, 30075, Roswell",10.0,Above average,public,30075,15.0,4
1,0,1108,82,Gwinnett County School District,Lawrenceville,34.044102,-83.924065,Fort Daniel Elementary School,PK-5,"1725 Auburn Road, 30019, Dacula",10.0,Above average,public,30019,14.0,5
2,0,1111,82,Gwinnett County School District,Lawrenceville,33.996933,-83.937210,Dyer Elementary School,PK-5,"1707 Hurricane Shoals Road Northeast, 30019, D...",10.0,Above average,public,30019,15.0,3
3,0,1503,124,Oconee County Schools,Watkinsville,33.914272,-83.506477,Malcom Bridge Elementary School,K-5,"2600 Malcom Bridge Road, 30622, Bogart",10.0,Above average,public,30622,16.0,4
4,0,3690,44,Columbia County School District,Evans,33.573559,-82.107941,River Ridge Elementary School,PK-5,"4109 Mullikin Road, 30809, Evans",10.0,Above average,public,30809,15.0,4


In [21]:
print("Summary statistics of elem_school data: ")
elem_school.describe()

Summary statistics of elem_school data: 


,Unnamed: 0,id,districtID,lat,long,rating,zipcode,studentsPerTeacher,parentRating
count,2719.0,2719.000000,2719.000000,2719.000000,2719.000000,1282.000000,2719.000000,1385.000000,2719.000000
mean,0.0,3531.301949,40.006252,33.351058,-83.879605,5.247270,30661.876793,14.462816,2.826775
std,0.0,2782.309431,53.992392,0.977093,1.005372,1.926157,1079.140650,2.659417,1.989432
min,0.0,2.000000,0.000000,25.530533,-85.548866,1.000000,30002.000000,1.000000,0.000000
25%,0.0,1286.000000,0.000000,32.870861,-84.481625,4.000000,30114.000000,13.000000,0.000000
50%,0.0,2478.000000,4.000000,33.708672,-84.212189,5.000000,30324.000000,15.000000,4.000000
75%,0.0,6225.500000,73.000000,33.963186,-83.648273,7.000000,31005.500000,16.000000,4.000000
max,0.0,9053.000000,270.000000,34.977684,-80.488266,10.000000,39897.000000,47.000000,5.000000


In [22]:
print("Missing values in elem_school data: ")
elem_school.isnull().sum()

Missing values in elem_school data: 


Unnamed: 0               0
id                       0
districtID               0
districtName          1309
districtCity          1309
lat                      0
long                     0
name                     0
gradeLevels              0
address                  0
rating                1437
ratingScale           1437
schoolType               0
zipcode                  0
studentsPerTeacher    1334
parentRating             0
dtype: int64

### mid_school

In [23]:
print("Shape of mid_school data: ", mid_school.shape)
mid_school.head()

Shape of mid_school data:  (1441, 16)


,Unnamed: 0,id,districtID,districtName,districtCity,lat,long,name,gradeLevels,address,rating,ratingScale,schoolType,zipcode,studentsPerTeacher,parentRating
0,0,2753,58,Dougherty County School District,Albany,31.573450,-84.249695,Robert A. Cross Middle Magnet,6-8,"324 Lockett Station Road, 31721, Albany",10.0,Above average,public,31721,15.0,3
1,0,931,69,Fayette County School District,Fayetteville,33.342094,-84.519585,Rising Starr Middle School,6-8,"183 Panther Path, 30215, Fayetteville",10.0,Above average,public,30215,15.0,3
2,0,1012,73,Fulton County School District,Atlanta,34.067436,-84.231171,Webb Bridge Middle School,6-8,"4455 Webb Bridge Road, 30005, Alpharetta",9.0,Above average,public,30005,14.0,5
3,0,1033,73,Fulton County School District,Atlanta,34.052856,-84.209183,Taylor Road Middle School,6-8,"5150 Taylor Rd, 30022, Alpharetta",9.0,Above average,public,30022,15.0,3
4,0,1614,138,Richmond County School District,Augusta,33.473846,-81.975899,Davidson Magnet School,6-12,"615 12th Street, 30901, Augusta",9.0,Above average,public,30901,18.0,5


In [24]:
print("Summary statistics of mid_school edata: ")
mid_school.describe()

Summary statistics of mid_school edata: 


,Unnamed: 0,id,districtID,lat,long,rating,zipcode,studentsPerTeacher,parentRating
count,1441.0,1441.000000,1441.000000,1441.000000,1441.000000,578.000000,1441.000000,619.000000,1441.000000
mean,0.0,3801.254684,36.343511,33.289537,-83.825294,5.204152,30735.346287,14.752827,2.508675
std,0.0,2716.740642,55.350418,1.001791,1.046561,1.928492,1224.062897,3.678236,1.930051
min,0.0,1.000000,0.000000,25.530533,-85.518494,1.000000,30002.000000,1.000000,0.000000
25%,0.0,1917.000000,0.000000,32.645199,-84.465767,4.000000,30127.000000,14.000000,0.000000
50%,0.0,2863.000000,0.000000,33.643238,-84.196259,5.000000,30342.000000,15.000000,3.000000
75%,0.0,6375.000000,66.000000,33.949112,-83.504730,7.000000,31054.000000,16.000000,4.000000
max,0.0,9047.000000,267.000000,34.981781,-80.488266,10.000000,39897.000000,37.000000,5.000000


In [25]:
print("Missing values in mid_school data: ")
mid_school.isnull().sum()

Missing values in mid_school data: 


Unnamed: 0              0
id                      0
districtID              0
districtName          808
districtCity          808
lat                     0
long                    0
name                    0
gradeLevels             0
address                 0
rating                863
ratingScale           863
schoolType              0
zipcode                 0
studentsPerTeacher    822
parentRating            0
dtype: int64

### high_school

In [26]:
print("Shape of high_school data: ", high_school.shape)
crime.head()

Shape of high_school data:  (1104, 16)


,Unnamed: 0,census_state_abbr,census_zcta5_geoid,census_cbsa_geoid_count,census_cbsa_geoid_list,overall_crime_grade,violent_crime_grade,violent_crime_total_rate,violent_crime_assault_rate,violent_crime_robbery_rate,...,property_crime_arson_rate,other_crime_grade,other_crime_total_rate,other_crime_kidnapping_rate,other_crime_drug_crimes_rate,other_crime_vandalism_rate,other_crime_identity_theft_rate,other_crime_animal_cruelty_rate,crime_safety_paragraph,interpreting_crime_map_paragraph
0,13719,GA,30002,1,12060,D-,C,3.106,1.7060,1.0670,...,0.1075,B+,5.987,0.1236,0.9456,3.981,0.8824,0.0542,The D- grade means the rate of crime is much h...,"When looking at the crime map for 30002, remem..."
1,13720,GA,30004,1,12060,C-,A-,1.553,0.9140,0.3858,...,0.1017,A+,3.171,0.0404,0.7863,1.524,0.8159,0.0042,The C- grade means the rate of crime is slight...,"When looking at the crime map for 30004, remem..."
2,13721,GA,30005,1,12060,B+,A-,1.480,0.7977,0.4288,...,0.1118,A+,2.709,0.0512,0.3942,1.394,0.8653,0.0041,The B+ grade means the rate of crime is lower ...,"When looking at the crime map for 30005, remem..."
3,13722,GA,30008,1,12060,D+,C,2.910,1.2040,1.3940,...,0.0946,B-,7.648,0.0883,0.9090,5.777,0.8214,0.0522,The D+ grade means the rate of crime is higher...,"When looking at the crime map for 30008, remem..."
4,13723,GA,30009,1,12060,D-,A-,1.611,1.0460,0.2950,...,0.1224,A,4.314,0.0453,0.8082,2.565,0.8910,0.0044,The D- grade means the rate of crime is much h...,"When looking at the crime map for 30009, remem..."


In [27]:
print("Summary statistics of high_school data: ")
high_school.describe()

Summary statistics of high_school data: 


,Unnamed: 0,id,districtID,lat,long,rating,zipcode,studentsPerTeacher,parentRating
count,1104.0,1104.000000,1104.000000,1104.000000,1104.000000,447.000000,1104.000000,488.000000,1104.000000
mean,0.0,3749.912138,36.835145,33.269079,-83.813699,5.000000,30763.145833,15.899590,2.531703
std,0.0,2785.346754,53.670818,1.023405,1.035285,1.974047,1273.091013,5.763669,1.933831
min,0.0,3.000000,0.000000,25.530533,-85.508987,1.000000,30002.000000,1.000000,0.000000
25%,0.0,1800.500000,0.000000,32.571535,-84.449206,4.000000,30135.000000,15.000000,0.000000
50%,0.0,2573.500000,0.000000,33.612899,-84.164024,5.000000,30349.500000,16.000000,3.000000
75%,0.0,6492.250000,70.000000,33.955349,-83.475910,6.000000,31063.000000,18.000000,4.000000
max,0.0,9047.000000,268.000000,34.977684,-80.488266,10.000000,39886.000000,78.000000,5.000000


In [28]:
print("Missing values in high_school data: ")
high_school.isnull().sum()

Missing values in high_school data: 


Unnamed: 0              0
id                      0
districtID              0
districtName          601
districtCity          601
lat                     0
long                    0
name                    0
gradeLevels             0
address                 0
rating                657
ratingScale           657
schoolType              0
zipcode                 0
studentsPerTeacher    616
parentRating            0
dtype: int64

### GA_listing

In [29]:
print("Shape of GA_listing data: ", GA_listing.shape)
crime.head()

Shape of GA_listing data:  (31064, 22)


,Unnamed: 0,census_state_abbr,census_zcta5_geoid,census_cbsa_geoid_count,census_cbsa_geoid_list,overall_crime_grade,violent_crime_grade,violent_crime_total_rate,violent_crime_assault_rate,violent_crime_robbery_rate,...,property_crime_arson_rate,other_crime_grade,other_crime_total_rate,other_crime_kidnapping_rate,other_crime_drug_crimes_rate,other_crime_vandalism_rate,other_crime_identity_theft_rate,other_crime_animal_cruelty_rate,crime_safety_paragraph,interpreting_crime_map_paragraph
0,13719,GA,30002,1,12060,D-,C,3.106,1.7060,1.0670,...,0.1075,B+,5.987,0.1236,0.9456,3.981,0.8824,0.0542,The D- grade means the rate of crime is much h...,"When looking at the crime map for 30002, remem..."
1,13720,GA,30004,1,12060,C-,A-,1.553,0.9140,0.3858,...,0.1017,A+,3.171,0.0404,0.7863,1.524,0.8159,0.0042,The C- grade means the rate of crime is slight...,"When looking at the crime map for 30004, remem..."
2,13721,GA,30005,1,12060,B+,A-,1.480,0.7977,0.4288,...,0.1118,A+,2.709,0.0512,0.3942,1.394,0.8653,0.0041,The B+ grade means the rate of crime is lower ...,"When looking at the crime map for 30005, remem..."
3,13722,GA,30008,1,12060,D+,C,2.910,1.2040,1.3940,...,0.0946,B-,7.648,0.0883,0.9090,5.777,0.8214,0.0522,The D+ grade means the rate of crime is higher...,"When looking at the crime map for 30008, remem..."
4,13723,GA,30009,1,12060,D-,A-,1.611,1.0460,0.2950,...,0.1224,A,4.314,0.0453,0.8082,2.565,0.8910,0.0044,The D- grade means the rate of crime is much h...,"When looking at the crime map for 30009, remem..."


In [30]:
print("Summary statistics of GA_listing data: ")
GA_listing.describe()

Summary statistics of GA_listing data: 


,Unnamed: 0,latitude,longitude,beds,baths_full,baths_half,square_footage,lot_size,year_built,special_features,unit_count,price,transaction_type,listing_status,listing_special_features
count,31064.000000,31064.000000,31064.000000,14167.000000,13978.000000,5855.000000,1.267800e+04,28387.000000,16620.000000,31064.000000,219.000000,3.106400e+04,31064.0,31064.000000,31064.000000
mean,15531.500000,33.667087,-83.793974,3.600692,2.513950,1.103501,3.216709e+03,19.311093,1985.574489,2.511879,3.529680,4.942250e+05,1.0,0.958891,2.511879
std,8967.548717,0.957898,0.996623,1.366697,1.253564,0.528206,2.546856e+04,657.400055,34.329650,34.077367,6.351536,1.332321e+06,0.0,0.198545,34.077367
min,0.000000,30.361079,-85.522663,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1700.000000,0.000000,1.000000,1.000000e+00,1.0,0.000000,0.000000
25%,7765.750000,33.302195,-84.428492,3.000000,2.000000,1.000000,1.450000e+03,0.000000,1967.000000,0.000000,2.000000,9.000000e+04,1.0,1.000000,0.000000
50%,15531.500000,33.825564,-84.015629,3.000000,2.000000,1.000000,2.052000e+03,1.000000,1994.000000,0.000000,2.000000,2.850000e+05,1.0,1.000000,0.000000
75%,23297.250000,34.323055,-83.461731,4.000000,3.000000,1.000000,2.952000e+03,3.000000,2010.000000,0.000000,4.000000,5.229468e+05,1.0,1.000000,0.000000
max,31063.000000,34.995653,-80.841980,71.000000,56.000000,10.000000,2.090880e+06,73616.000000,2023.000000,514.000000,89.000000,1.626270e+08,1.0,1.000000,514.000000


In [31]:
print("Missing values in GA_listing data: ")
GA_listing.isnull().sum()

Missing values in GA_listing data: 


Unnamed: 0                      0
latitude                        0
longitude                       0
full_street_address             0
city                            0
county_name                     0
beds                        16897
baths_full                  17086
baths_half                  25209
square_footage              18386
lot_size                     2677
year_built                  14444
details                         0
special_features                0
unit_count                  30845
price                           0
transaction_type                0
listing_status                  0
listing_special_features        0
census_state_name               0
census_county_name              0
zip                             0
dtype: int64